# Определение места под нефтянную скважину. Описание проекта

**Задача:** определить, где бурить новую скважину.

**Шаги для выбора локации:**
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Нужно построить модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap.

In [124]:
# Импорт стандартной библиотеки
import pandas as pd
import numpy as np
import random

# Сторонний импорт
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import LinearRegression
from sklearn.metrics         import mean_squared_error

import warnings
warnings.filterwarnings('ignore')


In [125]:
df_list = {'data':[geo_data_0, geo_data_1, geo_data_2], 
           'name':['geo_data_0', 'geo_data_1', 'geo_data_2']}
df = geo_data_0.append(geo_data_1).append(geo_data_2)
display(df)
print(df.info())
display(df.describe())
print()
print('Количество дубликатов:', df.duplicated().sum())


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       300000 non-null  object 
 1   f0       300000 non-null  float64
 2   f1       300000 non-null  float64
 3   f2       300000 non-null  float64
 4   product  300000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 13.7+ MB
None


,f0,f1,f2,product
count,300000.000000,300000.000000,300000.000000,300000.000000
mean,0.547913,-1.516172,2.497439,85.441667
std,5.316634,3.900220,2.916502,46.519494
min,-31.609576,-26.358598,-12.088328,0.000000
25%,-1.003037,-2.478939,0.648213,52.666629
50%,0.347934,-0.229632,2.491215,84.038886
75%,1.755365,0.678562,4.344943,124.174086
max,29.421755,18.734063,16.739402,190.029838



Количество дубликатов: 0


**Описание данных:**
- `id`: *уникальный идентификатор скважины;*
- `f0, f1, f2`: *три признака точек;*
- `product`: *объём запасов в скважине (тыс. баррелей).*

### Анализ данных
- в таблице нет дубликатов.
- нет пропусков.
- нет категориальных признаков.
- визуально, значения describe в пределах нормы.
- столбец `id` может отрицательно повлять на качество обучения модели.
- определяем `product` как целевой. Типа данных: float

# Исследование данных
**Условия задачи:**
- Для обучения модели использовать только линейную регрессию.

In [126]:
# Функция предобработки с выборками.  
def model(data):      
    # Разбьем данные на обучающую и валидационную выборки в соотношении 75:25
    y = data['product'].astype('int')
    X = data.loc[:,'f0':'f2']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12345)
    # Обучим модель и сделаем предсказание на валидационной выборке
    model = LinearRegression().fit(X_train, y_train)
    predicted_valid = model.predict(X_valid)
    rmse = (mean_squared_error(y_valid, predicted_valid) ** .5)
    
    valid_data_plus = ((X_valid)
                       .assign(y_valid = y_valid)
                       .assign(predicted_valid = predicted_valid.astype('int'))
                       .join(df_list['data'][0]['id']))
    
    result = {'rmse':rmse, 'predicted_valid':predicted_valid,
              'X_train':X_train,'X_valid':X_valid,
              'y_train':y_train, 'y_valid':y_valid,
              'valid_data_plus':valid_data_plus}
    
    return result


**Условия задачи:**
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [127]:
# Подготовка к расчёту прибыли
# Сохраним ключевые значения для расчётов
valid_data_geo0 = model(df_list['data'][0])['valid_data_plus']
valid_data_geo1 = model(df_list['data'][1])['valid_data_plus']
valid_data_geo2 = model(df_list['data'][2])['valid_data_plus']
valid_data = {'data':[valid_data_geo0, valid_data_geo1, valid_data_geo2],
              'name':['valid_data_geo0', 'valid_data_geo1', 'valid_data_geo2']}


In [128]:
# ключевые значения для расчётов
budget = 10000000000 # Бюджет на разработку скважин в регионе — 10 млрд рублей.
price = 450 # цена за баррель 450 рублей. В таблице указаны тысячи баррелей
wells = 100000 # Количество скважин в каждом регионе

well = int(budget/wells) # бюджет на одну скважину
volume = int(well/price) # объем сырья (баррелей) для безубыточной разработки новой скважины. Расходы на скважину
print()
print('Бюджет на одну скважину',well/1000,'тыс. рублей. Объем сырья для безубыточной разработки новой скважины:',volume/1000,'тыс. баррелей.')
print()
for i in range(3):
    volume_per_well = int(valid_data['data'][i]['y_valid'].mean())
    print("Средний запас сырья в регионе",valid_data['name'][i]+":",volume_per_well, 'тыс. баррелей')
    
print()



Бюджет на одну скважину 100.0 тыс. рублей. Объем сырья для безубыточной разработки новой скважины: 0.222 тыс. баррелей.

Средний запас сырья в регионе valid_data_geo0: 91 тыс. баррелей
Средний запас сырья в регионе valid_data_geo1: 68 тыс. баррелей
Средний запас сырья в регионе valid_data_geo2: 94 тыс. баррелей



In [129]:
# для выбора 200 лучших скважин используем предсказанные значения, а для подсчета прибыли настоящие значения соответствующих ячеек.
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False) # сортируем по прогнозу
    selected = (target[probs_sorted.index][:count]) # выбираем настоящие значения по прогнозу    
    return 450000 * selected.sum() - budget # считаем прибыль. Отнимаем весь бюджет


In [130]:
# Бутстреп для моделей
# В предыдущих шагах было получено 25000 настоящих и предсказанных значений для каждого региона.
#
# На каждой итерации бутстрапа нужно:
# Принимать 500 настоящих и 500 предсказанных значений для каждого региона - выбрать 500 случайных скважин из 25000.
# Посчитать по ним прибыль и записать полученной значение в массив прибылей. В результате получится 1000 значений прибыли для каждого региона.
#
# PS: Для выбора 200 лучших скважин нужно использовать предсказанные значения, а для подсчета прибыли настоящие значения соответствующих ячеек.
#
# Затем посчитаем 3 статистики:
#- Среднюю прибыль
#- Интервал с помощью метода квантилей
#- Риск убытков – долю случаев, в которых прибыль отрицательна.

def sampling(data_index):
    data = valid_data['data'][data_index]
    target = data['y_valid']    
    probabilities = data['predicted_valid']
    
    name = valid_data['name'][data_index] # Название региона
    repeat = 1000 # Количество итераций
    count = 200   # Число лучших скважин
    
    state = np.random.RandomState(12345)
    values = []
    for i in range(repeat):
        
        target_subsample = target.sample(n=500,replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        res = revenue(target_subsample, probs_subsample, count)
        values.append(res)

    values = pd.Series(values)
    upper = values.quantile(.975)
    lower = values.quantile(.025)
    mean = values.mean()
    risk = (len(values[values<0])/len(values))*100
    
    print()
    print("Регион:          ", name)
    print("Средняя выручка:   ", int(mean),"рублей")
    print("Квантиль  97,5%:   ", int(upper),"рублей")
    print("Квантиль   2,5%:   ", int(lower),"рублей")
    print("Риск убытков:      ", risk,"% из 100%")
    print("---------------------")


In [131]:
# Расчёт прибыли. Бутстреп для моделей. Считаем 3 статистики:
for i in range(3):
    sampling(i)



Регион:           valid_data_geo0
Средняя выручка:    379562300 рублей
Квантиль  97,5%:    889572500 рублей
Квантиль   2,5%:    -142772500 рублей
Риск убытков:       8.4 % из 100%
---------------------

Регион:           valid_data_geo1
Средняя выручка:    446605550 рублей
Квантиль  97,5%:    855361250 рублей
Квантиль   2,5%:    3927500 рублей
Риск убытков:       2.3 % из 100%
---------------------

Регион:           valid_data_geo2
Средняя выручка:    391044500 рублей
Квантиль  97,5%:    926945000 рублей
Квантиль   2,5%:    -174317500 рублей
Риск убытков:       8.9 % из 100%
---------------------


## Расчет прибыли. Риск убытков
- Риск убытков geo_data_0 и geo_data_2 выше 2,5%, отбрасываем эти регионы.
- Риск убытков в регионе geo_data_1 ниже 2,5%. Оставляем только его.
- Так же у geo_data_1 самая большая средняя выручка.